<a href="https://colab.research.google.com/github/nonoperry/Engie-Coding-Challenge/blob/main/Engie.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

# Data

In [85]:
data = {
    "load": 480,
    "fuels": {
        "gas(euro/MWh)": 13.4,
        "kerosine(euro/MWh)": 50.8,
        "co2(euro/ton)": 20,
        "wind(%)": 60
    },
    "powerplants": [
        {
            "name": "gasfiredbig1",
            "type": "gasfired",
            "efficiency": 0.53,
            "pmin": 100,
            "pmax": 460
        },
        {
            "name": "gasfiredbig2",
            "type": "gasfired",
            "efficiency": 0.53,
            "pmin": 100,
            "pmax": 460
        },
        {
            "name": "gasfiredsomewhatsmaller",
            "type": "gasfired",
            "efficiency": 0.37,
            "pmin": 40,
            "pmax": 210
        },
        {
            "name": "tj1",
            "type": "turbojet",
            "efficiency": 0.3,
            "pmin": 0,
            "pmax": 16
        },
        {
            "name": "windpark1",
            "type": "windturbine",
            "efficiency": 1,
            "pmin": 0,
            "pmax": 150
        },
        {
            "name": "windpark2",
            "type": "windturbine",
            "efficiency": 1,
            "pmin": 0,
            "pmax": 36
        }
    ]
}

In [87]:
data = {
  "load": 480,
  "fuels":
  {
    "gas(euro/MWh)": 13.4,
    "kerosine(euro/MWh)": 50.8,
    "co2(euro/ton)": 20,
    "wind(%)": 0
  },
  "powerplants": [
    {
      "name": "gasfiredbig1",
      "type": "gasfired",
      "efficiency": 0.53,
      "pmin": 100,
      "pmax": 460
    },
    {
      "name": "gasfiredbig2",
      "type": "gasfired",
      "efficiency": 0.53,
      "pmin": 100,
      "pmax": 460
    },
    {
      "name": "gasfiredsomewhatsmaller",
      "type": "gasfired",
      "efficiency": 0.37,
      "pmin": 40,
      "pmax": 210
    },
    {
      "name": "tj1",
      "type": "turbojet",
      "efficiency": 0.3,
      "pmin": 0,
      "pmax": 16
    },
    {
      "name": "windpark1",
      "type": "windturbine",
      "efficiency": 1,
      "pmin": 0,
      "pmax": 150
    },
    {
      "name": "windpark2",
      "type": "windturbine",
      "efficiency": 1,
      "pmin": 0,
      "pmax": 36
    }
  ]
}

In [72]:
data = {
  "load": 910,
  "fuels":
  {
    "gas(euro/MWh)": 13.4,
    "kerosine(euro/MWh)": 50.8,
    "co2(euro/ton)": 20,
    "wind(%)": 60
  },
  "powerplants": [
    {
      "name": "gasfiredbig1",
      "type": "gasfired",
      "efficiency": 0.53,
      "pmin": 100,
      "pmax": 460
    },
    {
      "name": "gasfiredbig2",
      "type": "gasfired",
      "efficiency": 0.53,
      "pmin": 100,
      "pmax": 460
    },
    {
      "name": "gasfiredsomewhatsmaller",
      "type": "gasfired",
      "efficiency": 0.37,
      "pmin": 40,
      "pmax": 210
    },
    {
      "name": "tj1",
      "type": "turbojet",
      "efficiency": 0.3,
      "pmin": 0,
      "pmax": 16
    },
    {
      "name": "windpark1",
      "type": "windturbine",
      "efficiency": 1,
      "pmin": 0,
      "pmax": 150
    },
    {
      "name": "windpark2",
      "type": "windturbine",
      "efficiency": 1,
      "pmin": 0,
      "pmax": 36
    }
  ]
}

# Code

In [84]:
#GOOOOOOOOD

import gurobipy as gp
from gurobipy import GRB

model = gp.Model("PowerProd")

produced = {}

for plant in data['powerplants']:
    produced[plant['name']] = model.addVar(vtype=GRB.CONTINUOUS, name=plant['name'])

# Binary variables to indicate whether each power plant is on or off
on_off = {plant['name']: model.addVar(vtype=GRB.BINARY, name=f"{plant['name']}_on") for plant in data['powerplants']}

total_cost = gp.quicksum(
    (produced[plant['name']] / plant['efficiency']) * data['fuels']['gas(euro/MWh)']
    + (produced[plant['name']] / plant['efficiency']) * data['fuels']['co2(euro/ton)'] * 0.3
    if plant['type'] == 'gasfired'
    else (produced[plant['name']] / plant['efficiency']) * data['fuels']['kerosine(euro/MWh)']
    if plant['type'] == 'turbojet'
    else 0
    for plant in data['powerplants']
)

model.setObjective(total_cost, GRB.MINIMIZE)

# Add constraints for each power plant's capacity and on/off status
for plant in data["powerplants"]:
    model.addConstr(produced[plant["name"]] >= plant["pmin"] * on_off[plant["name"]])
    model.addConstr(produced[plant["name"]] <= plant["pmax"] * on_off[plant["name"]])

# Add constraint for total power produced to meet the load
model.addConstr(gp.quicksum(produced.values()) >= data["load"])

# Calculate the maximum wind power generation based on wind percentage
wind_percentage = data['fuels']['wind(%)'] / 100

# Add constraint for each wind turbine's capacity, considering wind percentage
for plant in data["powerplants"]:
    if plant["type"] == "windturbine":
        max_wind_power = plant["pmax"] * wind_percentage
        model.addConstr(produced[plant["name"]] <= max_wind_power)

# Optimize the model
model.optimize()

# Print results
if model.status == GRB.OPTIMAL:
    print("Optimal solution found")
    for plant in produced:
        print(f"Power plant: {plant}, Power produced: {produced[plant].x} MW")
    print(f"Total cost: {model.objVal} euros")
else:
    print("No solution found")


Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (linux64)

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 15 rows, 12 columns and 29 nonzeros
Model fingerprint: 0xbbfef59f
Variable types: 6 continuous, 6 integer (6 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+02]
  Objective range  [4e+01, 2e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+01, 9e+02]
Presolve removed 15 rows and 12 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.07 seconds (0.00 work units)
Thread count was 1 (of 2 available processors)

Solution count 1: 29224.5 

Optimal solution found (tolerance 1.00e-04)
Best objective 2.922445283019e+04, best bound 2.922445283019e+04, gap 0.0000%
Optimal solution found
Power plant: gasfiredbig1, Power produced: 338.4 MW
Power plant: gasfiredbig2, Power produced

In [86]:
import gurobipy as gp
from gurobipy import GRB

model = gp.Model("PowerProd")

produced = {}

for plant in data['powerplants']:
    produced[plant['name']] = model.addVar(vtype=GRB.CONTINUOUS, name=plant['name'])

# Binary variables to indicate whether each power plant is on or off
on_off = {plant['name']: model.addVar(vtype=GRB.BINARY, name=f"{plant['name']}_on") for plant in data['powerplants']}

total_cost = gp.quicksum(
    (produced[plant['name']] / plant['efficiency']) * data['fuels']['gas(euro/MWh)']
    + (produced[plant['name']] / plant['efficiency']) * data['fuels']['co2(euro/ton)'] * 0.3
    if plant['type'] == 'gasfired'
    else (produced[plant['name']] / plant['efficiency']) * data['fuels']['kerosine(euro/MWh)']
    if plant['type'] == 'turbojet'
    else 0
    for plant in data['powerplants']
)

model.setObjective(total_cost, GRB.MINIMIZE)

# Add constraints for each power plant's capacity and on/off status
for plant in data["powerplants"]:
    model.addConstr(produced[plant["name"]] >= plant["pmin"] * on_off[plant["name"]])
    model.addConstr(produced[plant["name"]] <= plant["pmax"] * on_off[plant["name"]])

# Add constraint for total power produced to meet the load
model.addConstr(gp.quicksum(produced.values()) >= data["load"])

# Calculate the maximum wind power generation based on wind percentage
wind_percentage = data['fuels']['wind(%)'] / 100

# Add constraint for each wind turbine's capacity, considering wind percentage
for plant in data["powerplants"]:
    if plant["type"] == "windturbine":
        max_wind_power = plant["pmax"] * wind_percentage
        model.addConstr(produced[plant["name"]] <= max_wind_power)

# Optimize the model
model.optimize()

# Print results
if model.status == GRB.OPTIMAL:
    print("Optimal solution found")
    for plant in produced:
        print(f"Power plant: {plant}, Power produced: {produced[plant].x} MW")
    print(f"Total cost: {model.objVal} euros")
else:
    print("No solution found")


Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (linux64)

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 15 rows, 12 columns and 29 nonzeros
Model fingerprint: 0xc8de175e
Variable types: 6 continuous, 6 integer (6 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+02]
  Objective range  [4e+01, 2e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+01, 5e+02]
Presolve removed 8 rows and 5 columns
Presolve time: 0.00s
Presolved: 7 rows, 7 columns, 18 nonzeros
Variable types: 4 continuous, 3 integer (3 binary)
Found heuristic solution: objective 13484.830189

Root relaxation: cutoff, 0 iterations, 0.00 seconds (0.00 work units)

Explored 1 nodes (0 simplex iterations) in 0.02 seconds (0.00 work units)
Thread count was 2 (of 2 available processors)

Solution count 1: 13484.8 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.34848

In [88]:
import gurobipy as gp
from gurobipy import GRB

model = gp.Model("PowerProd")

produced = {}

for plant in data['powerplants']:
    produced[plant['name']] = model.addVar(vtype=GRB.CONTINUOUS, name=plant['name'])

# Binary variables to indicate whether each power plant is on or off
on_off = {plant['name']: model.addVar(vtype=GRB.BINARY, name=f"{plant['name']}_on") for plant in data['powerplants']}

total_cost = gp.quicksum(
    (produced[plant['name']] / plant['efficiency']) * data['fuels']['gas(euro/MWh)']
    + (produced[plant['name']] / plant['efficiency']) * data['fuels']['co2(euro/ton)'] * 0.3
    if plant['type'] == 'gasfired'
    else (produced[plant['name']] / plant['efficiency']) * data['fuels']['kerosine(euro/MWh)']
    if plant['type'] == 'turbojet'
    else 0
    for plant in data['powerplants']
)

model.setObjective(total_cost, GRB.MINIMIZE)

# Add constraints for each power plant's capacity and on/off status
for plant in data["powerplants"]:
    model.addConstr(produced[plant["name"]] >= plant["pmin"] * on_off[plant["name"]])
    model.addConstr(produced[plant["name"]] <= plant["pmax"] * on_off[plant["name"]])

# Add constraint for total power produced to meet the load
model.addConstr(gp.quicksum(produced.values()) >= data["load"])

# Calculate the maximum wind power generation based on wind percentage
wind_percentage = data['fuels']['wind(%)'] / 100

# Add constraint for each wind turbine's capacity, considering wind percentage
for plant in data["powerplants"]:
    if plant["type"] == "windturbine":
        max_wind_power = plant["pmax"] * wind_percentage
        model.addConstr(produced[plant["name"]] <= max_wind_power)

# Optimize the model
model.optimize()

# Print results
if model.status == GRB.OPTIMAL:
    print("Optimal solution found")
    for plant in produced:
        print(f"Power plant: {plant}, Power produced: {produced[plant].x} MW")
    print(f"Total cost: {model.objVal} euros")
else:
    print("No solution found")


Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (linux64)

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 15 rows, 12 columns and 29 nonzeros
Model fingerprint: 0xcac82e3f
Variable types: 6 continuous, 6 integer (6 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+02]
  Objective range  [4e+01, 2e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [5e+02, 5e+02]
Presolve removed 8 rows and 5 columns
Presolve time: 0.00s
Presolved: 7 rows, 7 columns, 18 nonzeros
Variable types: 0 continuous, 7 integer (3 binary)
Found heuristic solution: objective 17569.811321

Root relaxation: cutoff, 0 iterations, 0.00 seconds (0.00 work units)

Explored 1 nodes (0 simplex iterations) in 0.04 seconds (0.00 work units)
Thread count was 2 (of 2 available processors)

Solution count 1: 17569.8 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.75698